In [1]:
from aliquoting import *
from distance import DirectedCopyNumberDistanceLinear

In [2]:
import numpy as np
import pandas as pd
from timeit import default_timer as timer
from tqdm.notebook import tqdm, trange

# Functions for generating simulated inputs

In [3]:
def get_cnt(n, num):
    """Generate a random CNT of num operations,
    for profiles of size n"""
    for _ in range(num):
        length = np.random.randint(1,n+1)
        start = np.random.randint(1, n+2-length)
        if np.random.rand() < 0.75: 
            # generate deletions more often (w.p. 3/4)
            yield (start, start+length-1, -1)
        else: # amplifications
            yield (start, start+length-1, +1)

In [4]:
class ZeroException(Exception):
    """Used for implementing rejection sampling"""
    pass

def apply_cnt(S, cnt):
    """Apply the transformation cnt to the profile S,
    making sure there are no zeroes in the final profile."""
    S2 = np.copy(S)
    for (start,end,w) in cnt:
        if w == -1:
            if 1 in S2[start-1:end]:
                raise ZeroException()
            else:
                S2[start-1:end] -= 1
        else: # w == 1
            S2[start-1:end] += 1
    return S2

In [5]:
def gen_T(n, p, k, num):
    """Generate a random preduplication profile and CNT"""
    for _ in range(num):
        S = np.random.randint(1,6,size=(n,))
        pS = p * S
        T = None
        while True:
            try:
                cnt = list(get_cnt(n, k))
                T = apply_cnt(pS, cnt)
                break
            except ZeroException:
                # we don't want final profiles that have zeroes
                continue
        assert T is not None
        cnd = DirectedCopyNumberDistanceLinear(pS,T)
        yield (S, T, cnt, cnd)

# Generate and test simulated inputs 

In [6]:
outputs = []
np.random.seed(2021)

num = 100
for n in tqdm([100,200,300], desc='n'):
    for p in tqdm([2,3,4], desc='p'):
        for k in tqdm([5,10, 15], desc='k'):
            for S, T, cnt, cnd in tqdm(gen_T(n, p, k, num), total=num):
                if p >= 3:
                    start_time = timer()
                    T_tag,idx,mapping = contract_mod_p(T,p)
                    dist, S_tag = cnd_aliquoting_I(T_tag,p)[:2]
                    S_ = update_S(S_tag,T,idx,mapping,p)
                    end_time = timer()
                    T_runs = len(T_tag)
                else: # p == 2: use halving algorithm
                    start_time = timer()
                    dist, S_ = cnd_halving(T)
                    end_time = timer()
                    T_runs = odd_runs(T)
                time_elapsed = end_time - start_time
                predup_dist = DirectedCopyNumberDistanceLinear(S, S_)
                outputs.append({
                    'n': n,
                    'p': p,
                    'k': k,
                    'S': S,
                    'T': T,
                    'T_runs': T_runs,
                    'cnt': cnt,
                    'd(pS,T)': cnd,
                    'n2_time': time_elapsed,
                    'S_': S_,
                    'd(pS_,T)': dist,
                    'd(S_,S)': predup_dist,
                })

n:   0%|          | 0/3 [00:00<?, ?it/s]

p:   0%|          | 0/3 [00:00<?, ?it/s]

k:   0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

...

  0%|          | 0/100 [00:00<?, ?it/s]

k:   0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

In [7]:
df = pd.DataFrame.from_records(outputs)

In [8]:
df.to_pickle('outputs.pkl')